### Computing a 1D FES from umbrella sampling

Example illustrating the application of MBAR to compute a 1D free energy profile from an umbrella sampling simulation. The data represents an umbrella sampling simulation for the chi torsion of<br>
a valine sidechain in lysozyme L99A with benzene bound in the cavity.

Reference for data.
    D. L. Mobley, A. P. Graves, J. D. Chodera, A. C. McReynolds, B. K. Shoichet and K. A. Dill,
    "Predicting absolute ligand binding free energies to a simple model site,"
    Journal of Molecular Biology 371(4):1118-1134 (2007)
    http://dx.doi.org/10.1016/j.jmb.2007.06.002


In [ ]:
import numpy as np

In [ ]:
import pymbar  # multistate Bennett acceptance ratio
from pymbar import timeseries  # timeseries analysis
import matplotlib.pyplot as plt

Physical  Constants

In [ ]:
kB = 1.381e-23 * 6.022e23 / 1000.0  # Boltzmann constant in kJ/mol/K

In [ ]:
temperature = 300  # assume a single temperature -- can be overridden with data from center.dat

Parameters for the data collected. 

In [ ]:
K = 26  # number of umbrellas
N_max = 501  # maximum number of snapshots/simulation
T_k = np.ones(K, float) * temperature  # inital temperatures are all equal
beta = 1.0 / (kB * temperature)  # inverse temperature of simulations (in 1/(kJ/mol))
chi_min = -180.0  # min for free energy profile
chi_max = +180.0  # max for free energy profile
nbins = 36  # number of bins to use for the histogram version of the 1D free energy profile

Allocate storage for simulation data<br>
The array N_k[k] is the number of snapshots from umbrella simulation $k$

In [ ]:
N_k = np.zeros([K], dtype=int)
# K_k[k] is the spring constant (in kJ/mol/deg**2) for umbrella simulation k
K_k = np.zeros([K])
# chi0_k[k] is the spring center location (in deg) for umbrella simulation k
chi0_k = np.zeros([K])
# chi_kn[k,n] is the torsion angle (in deg) for snapshot n from umbrella simulation k
chi_kn = np.zeros([K, N_max])
# u_kn[k,n] is the reduced potential energy without umbrella restraints of snapshot n of umbrella simulation k
u_kn = np.zeros([K, N_max])
g_k = np.zeros([K])

Read in umbrella spring constants and centers from the provided data file. for this data set, the force constants are not all the same, but the temperatures are. 

In [ ]:
with open("data/centers.dat") as infile:
    lines = infile.readlines()

In [ ]:
for k in range(K):
    # Parse line k.
    line = lines[k]
    tokens = line.split()
    chi0_k[k] = float(tokens[0])  # spring center location (in deg)
    # spring constant (read in kJ/mol/rad**2, converted to kJ/mol/deg**2)
    K_k[k] = float(tokens[1]) * (np.pi / 180) ** 2
    if len(tokens) > 2:
        T_k[k] = float(tokens[2])  # temperature the kth simulation was run at.

In [ ]:
beta_k = 1.0 / (kB * T_k)  # beta factor for the different temperatures
different_temperatures = True
if min(T_k) == max(T_k):
    # if all the temperatures are the same, then we don't have to read in energies.
    different_temperatures = False

Let's print out the location and magnitude of the torsion biases

In [ ]:
for theta,k in zip(chi0_k, K_k):
    print(f'{theta:-9.1f}{k:10.4f}')

In [ ]:
xplot = np.linspace(chi_min,chi_max,1000)
for theta,k in zip(chi0_k, K_k):
    plt.plot(xplot,0.5*k*(xplot-theta)**2)
plt.xlim([chi_min,chi_max])
plt.ylim([0,3])
plt.show()

Read the simulation data, which is expressed as torsion angles.  We also compute the statistical inefficiency to be able to subsample the data using `timeseries.statistical_inefficiency`. 

In [ ]:
for k in range(K):
    # Read torsion angle data.
    filename = f"data/prod{k:d}_dihed.xvg"
    print(f"Reading {filename}...")
    n = 0
    with open(filename, "r") as infile:
        for line in infile:
            if line[0] != "#" and line[0] != "@":
                tokens = line.split()
                chi = float(tokens[1])  # torsion angle
                # wrap chi_kn to be within [-180,+180)
                while chi < -180.0:
                    chi += 360.0
                while chi >= +180.0:
                    chi -= 360.0
                chi_kn[k, n] = chi
                n += 1
    N_k[k] = n
    if different_temperatures:  # if different temperatures are specified the metadata file,
        # then we need the energies to compute the free energy profile
        # Read energies
        filename = f"data/prod{k:d}_energies.xvg"
        print(f"Reading {filename}...")
        n = 0
        with open(filename, "r") as infile:
            for line in infile:
                if line[0] != "#" and line[0] != "@":
                    tokens = line.split()
                    # reduced potential energy without umbrella restraint
                    u_kn[k, n] = beta_k[k] * (float(tokens[2]) - float(tokens[1]))
                    n += 1

    # Compute correlation times for potential energy and chi
    # timeseries.  If the temperatures differ, use energies to determine samples; otherwise, use the cosine of chi
    if different_temperatures:
        g_k[k] = timeseries.statistical_inefficiency(u_kn[k, :], u_kn[k, 0 : N_k[k]])
        print(f"Correlation time for set {k:5d} is {g_k[k]:10.3f}")
        indices = timeseries.subsample_correlated_data(u_kn[k, 0 : N_k[k]])
        print(f"Number of points for set {k:5d} is {len(indices):10d}")
    else:
        chi_radians = chi_kn[k, 0 : N_k[k]] / (180.0 / np.pi)
        g_cos = timeseries.statistical_inefficiency(np.cos(chi_radians))
        g_sin = timeseries.statistical_inefficiency(np.sin(chi_radians))
        print(f"g_cos = {g_cos:.1f} | g_sin = {g_sin:.1f}")
        g_k[k] = max(g_cos, g_sin)
        print(f"Correlation time for set {k:5d} is {g_k[k]:10.3f}")
        indices = timeseries.subsample_correlated_data(chi_radians, g=g_k[k])
        print(f"Number of points for set {k:5d} is {len(indices):10d}")
    # Subsample data.
    N_k[k] = len(indices)
    u_kn[k, 0 : N_k[k]] = u_kn[k, indices]
    chi_kn[k, 0 : N_k[k]] = chi_kn[k, indices]

In [ ]:
N_max = np.max(N_k)  # shorten the array size
# u_kln[k,l,n] is the reduced potential energy of snapshot n from umbrella simulation k evaluated at umbrella l
u_kln = np.zeros([K, K, N_max])
N = np.sum(N_k)

Set zero of `u_kn`. The choice is arbitrary; it doesn't affect the final answers, but can make it easier to read the data. 

In [ ]:
u_kn -= u_kn.min()

Compute the bin centers of a histogram.

In [ ]:
bin_center_i = np.zeros([nbins])
bin_edges = np.linspace(chi_min, chi_max, nbins + 1)
for i in range(nbins):
    bin_center_i[i] = 0.5 * (bin_edges[i] + bin_edges[i + 1])

We now evaluate reduced energies in all umbrellas.  We first compute the smallest distance from the data point to the bin center corresponding to the umbrella $k$, wrapping around the periodic image (this is the minimum data convention), then multiply it times the spring constant of state $k$.

Note that we don't need to know the energies of the simulations we collected - we only need to calculate what the bias potential is.  That's because the energy of the underlying simulation is the same in all cases, so it cancels out.  To reweight, we only need to know the value of the collective ariable.

In [ ]:
print("Evaluating reduced potential energies...")
for k in range(K):
    for n in range(N_k[k]):
        # Compute minimum-image torsion deviation from umbrella center l
        dchi = chi_kn[k, n] - chi0_k
        for l in range(K):
            if abs(dchi[l]) > 180.0:
                dchi[l] = 360.0 - abs(dchi[l])
        # Compute energy of snapshot n from simulation k in umbrella potential l
        u_kln[k, :, n] = u_kn[k, n] + beta_k[k] * (K_k / 2.0) * dchi**2

Initialize free energy profile with the data collected.  This is calling MBAR under the hood to determine the weights corresponding to each of the biased samples. 

In [ ]:
fes = pymbar.FES(u_kln, N_k, verbose=True)

Now extract the free energy at the bin centers.

In [ ]:
# convert to 1D representaion of the observables for generate_fes 
chi_n = pymbar.utils.kn_to_n(chi_kn, N_k=N_k)

# Compute free energy profile in unbiased potential (in units of kT).
histogram_parameters = {}
histogram_parameters["bin_edges"] = bin_edges
fes.generate_fes(u_kn, chi_n, fes_type="histogram", histogram_parameters=histogram_parameters)
results = fes.get_fes(bin_center_i, reference_point="from-lowest", uncertainty_method="analytical")
center_f_i = results["f_i"]
center_df_i = results["df_i"]

Write out free energy profile in as a function of bin, with uncertainty. 

In [ ]:
print("free energy profile (in units of kT), from histogramming")
print(f"{'bin':>8s} {'f':>8s} {'df':>8s}")
for i in range(nbins):
    print(f"{bin_center_i[i]:8.1f} {center_f_i[i]:8.3f} {center_df_i[i]:8.3f}")

And plot it, with estimated uncertainties, generated from analytical estimates of the uncertainty. 

In [ ]:
plt.errorbar(bin_center_i, center_f_i, yerr=center_df_i, label='histogram')
plt.title("Free energy surface for sidechain torsion")
plt.ylabel("free energy (units of kT)")
plt.xlabel("coordinate (degrees)")
plt.xlim(-180,180)
plt.legend()
plt.show()

Now compute the same PMF with kernel density estimates KDEs. We can compare the histogram free energy with the KDE free energy at each point.  This time we do need to specify a bandwidth, i.e, how big the Gaussian spread is for the kernel is at each point.  We'll pick half the width of whatever bin width was picked before; there are few possible options, though.

In [ ]:
kde_parameters = {}
kde_parameters["bandwidth"] = 0.5 * ((chi_max - chi_min) / nbins)
fes.generate_fes(u_kn, chi_n, fes_type="kde", kde_parameters=kde_parameters)
results = fes.get_fes(bin_center_i, reference_point="from-lowest")
# Write out free energy profile from KDE
kde_center_f_i = results["f_i"]
print("free energy profile (in units of kT), from KDE")
print(f"{'bin':>8s} {'f':>8s}")
for i in range(nbins):
    print(f"{bin_center_i[i]:8.1f} {kde_center_f_i[i]:8.3f}")

And plot both of them as a function of torsion angle (for now, we just use the errors for the histogram version); we don't really have an analytical error for the KDE.

In [ ]:
plt.plot(bin_center_i, kde_center_f_i, label='KDE')
plt.errorbar(bin_center_i, center_f_i, yerr=center_df_i, label='histogram')
plt.title("Free energy surface for sidechain torsion")
plt.ylabel("free energy surface (units of kT)")
plt.xlabel("coordinate")
plt.xlim(-180,180)
plt.legend()
plt.show()

They are almost, but not exactly the same.  Which is better?

If we are using a kernel density estimate, we don't have to compute at just the bin centers; we can actually print at any value, since we have learned a continuous function that best matches the data collected.

Let's evaluate the FES again.  This time, we will use bootstrap sampling to calculate the error, which means we will resample the collected data.  We'll turn off verbosity, as it will print the results many times.

How many points do we want to plot? Let's be ambitious!

In [ ]:
xplot = np.linspace(-180,180,1000)

Now we generate the free energy surface; note that will take longer, since we are solving for $N_{bootstraps}$ free energies.  Note that for good error estimates, we probably want to generate 50-ish bootstrap samples, but I'm using 20 to get to get reasonable close without waiting too long.  We'll also reduce the bandwidth down to 3 degrees, as that seems to give reasonably smooth results with the amount of data we have.

In [ ]:
kde_parameters['bandwidth']=3

In [ ]:
fes.generate_fes(u_kn, chi_n, fes_type="kde", kde_parameters=kde_parameters, n_bootstraps = 20)
results_kde = fes.get_fes(xplot, reference_point="from-lowest", uncertainty_method="bootstrap")
f_ik = results_kde["f_i"]
df_ik = results_kde["df_i"]

In [ ]:
plt.errorbar(xplot, f_ik, yerr=df_ik, label='kde', errorevery=10)
plt.title("Free energy surface for sidechain torsion")
plt.ylabel("free energy surface (units of kT)")
plt.xlabel("coordinate")
plt.xlim([-180,180])
plt.legend()
plt.show()

And we can compare to the histogram bootstrapped samples as well. First generate them.

In [ ]:
fes.generate_fes(u_kn, chi_n, fes_type="histogram", histogram_parameters=histogram_parameters, n_bootstraps = 20)
results_hist = fes.get_fes(xplot, reference_point="from-lowest", uncertainty_method="bootstrap")
f_ih = results_hist["f_i"]
df_ih = results_hist["df_i"]

Then plot the two together. 

In [ ]:
plt.errorbar(xplot, f_ik, yerr=df_ik, label='kde', errorevery=10)
plt.errorbar(xplot, f_ih, yerr=df_ih, label='histogram', errorevery=10)
plt.title("Free energy surface for sidechain torsion")
plt.ylabel("free energy surface (units of kT)")
plt.xlabel("coordinate")
plt.xlim(-180,180)
plt.legend()
plt.show()

Note that the histogram FES is actually displayed as a histogram, because that is what it is!